In [2]:
import numpy as np 
import pandas as pd
import os

from tabula import read_pdf
from tabulate import tabulate

In [3]:
# Define Dir with working raw datasets
os.getcwd()

pdf_dir = 'Raw Data/'
print(os.listdir(pdf_dir))

output_dir = 'Cleaned Data/'
print(os.listdir(output_dir))


['URS_PuertoRico.pdf', 'URS_VA.pdf', 'URS_Palau.pdf', 'URS_SC.pdf']
[]


In [5]:
# Define a function process_pdf to process raw pdf and save into a dataframe

def process_pdf(pdf_file, state_code): 

    ## Read table on page 4

    tables_pg4 = read_pdf(os.path.join(pdf_dir, pdf_file), pages=4, multiple_tables=True)
    
    ## Read tables[2] into a dataframe 
    df_total = pd.DataFrame(tables_pg4[2])

    ## Read tables from page 5 
    tables_pg5= read_pdf(os.path.join(pdf_dir,pdf_file), pages=5, multiple_tables=True )

    df_age = pd.DataFrame(tables_pg5[0])

    ## Only selecting specfic rows and columns that are associated with served patients & rates
    df_ageshort = df_age[["Unnamed: 0", "Unnamed: 1", "Unnamed: 5"]][4:13]

    ## Rename the df_ageshort dataframe by the column index
    df_2 = df_ageshort.rename(columns={df_ageshort.columns[0]: "Measure", 
                                       df_ageshort.columns[1]: "PatientsServed", 
                                       df_ageshort.columns[2]: "ServedRate"})



    ## Combine Total measure summary table with the age table 
    df_total = df_total.rename(columns= {df_total.columns[1]: "PatientsServed", 
                                         df_total.columns[2]: "ServedRate"})


    df_transformed = df_total.append(df_2)

    ## Add a column indicating the state 
    df_transformed['State'] = state_code

    return df_transformed


In [8]:
# Check if the output dir exists, if not, creat a dir
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)

# Create an empty list to save all the state-specific dataframe
all_transformed_data = []
# Loop through all PDF files in the PDF dir
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"): 
        state_code = pdf_file.split('_')[1].split('.')[0] # extract the state code from the file name
        df_transformed = process_pdf(pdf_file, state_code)
        print(df_transformed.head(20))
        all_transformed_data.append(df_transformed)
    
    

# Combine all states into a single dataframe
combined_df = pd.concat(all_transformed_data, ignore_index=True)

combined_df.head(5)


/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/783782525.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


           Gender PatientsServed ServedRate Percent.1            Measure  \
0          Female          40.0%    0 to 12     10.3%                NaN   
1            Male          60.0%   13 to 17     11.1%                NaN   
2           Other              -   18 to 20      2.8%                NaN   
3   Not Available              -   21 to 24      2.1%                NaN   
4             NaN            291          -       NaN               0-12   
5             NaN            313          -       NaN              13-17   
6             NaN             78          -       NaN              18-20   
7             NaN             59          -       NaN              21-24   
8             NaN            679          -       NaN              25-44   
9             NaN            938          -       NaN              45-64   
10            NaN            333          -       NaN              65-74   
11            NaN            127          -       NaN        75 and over   
12          

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/783782525.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


                                 Measure PatientsServed ServedRate State
0    Total Clients Served by SMHA System        123,936       14.3    VA
1   Clients Served in Community Settings        122,155       14.1    VA
2      Clients Served in State Hospitals          5,327        0.6    VA
4                                   0-12         15,802       11.9    VA
5                                  13-17         18,129       32.7    VA
6                                  18-20          5,825       16.9    VA
7                                  21-24          6,879       15.1    VA
8                                  25-44         39,398       16.8    VA
9                                  45-64         30,113       13.6    VA
10                                 65-74          6,130        7.2    VA
11                           75 and over          1,656        3.0    VA
12                     Age Not Available              4          -    VA


/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/783782525.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


                                 Measure PatientsServed ServedRate  State
0    Total Clients Served by SMHA System            443       24.6  Palau
1   Clients Served in Community Settings            443       24.6  Palau
2      Clients Served in State Hospitals              0        0.0  Palau
4                                   0-12             27          -  Palau
5                                  13-17             21          -  Palau
6                                  18-20             21          -  Palau
7                                  21-24             19          -  Palau
8                                  25-44            155          -  Palau
9                                  45-64            164          -  Palau
10                                 65-74             28          -  Palau
11                           75 and over              8          -  Palau
12                     Age Not Available              -          -  Palau
           Gender PatientsServed Serve

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/783782525.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


,Gender,PatientsServed,ServedRate,Percent.1,Measure,State
0,Female,40.0%,0 to 12,10.3%,NaN,PuertoRico
1,Male,60.0%,13 to 17,11.1%,NaN,PuertoRico
2,Other,-,18 to 20,2.8%,NaN,PuertoRico
3,Not Available,-,21 to 24,2.1%,NaN,PuertoRico
4,NaN,291,-,NaN,0-12,PuertoRico
